# Building Interpolating Polynomials

In [2]:
using Pkg
Pkg.activate(".")
using SymPy

  Activating project at `c:\Users\tim\Documents\2025examples\02-25`


In [3]:
a0,a1,a2,a3,a4 = symbols("a_0, a_1, a_2, a_3, a_4", real=true)
x1, x2, x3, x4, x5 = symbols("x1, x2, x3, x4, x5", real=true)
y1, y2, y3, y4, y5 = symbols("y1, y2, y3, y4, y5", real=true)
x,y = symbols("x,y", real=true)
X = [x1, x2, x3, x4, x5]
Y = [y1, y2, y3, y4, y5]
A = [a0, a1, a2, a3, a4];

## Naive construction
### $n = 2$

In [3]:
n = 2
y = a0 + a1*x

a_0 + a_1*x

In [4]:
sol = solve( [ -y1 + y(x=>x1), -y2 + y(x=>x2) ], 
[a0,a1] )

Dict{Sym{PyCall.PyObject}, Sym{PyCall.PyObject}} with 2 entries:
  a_0 => (x1*y2 - x2*y1)/(x1 - x2)
  a_1 => (y1 - y2)/(x1 - x2)

In [5]:
y(sol)

x*(y1 - y2)   x1*y2 - x2*y1
----------- + -------------
  x1 - x2        x1 - x2   

### $ n = 3 $ and onward

In [6]:
n = 3
y = A[1:n]' * x .^ (0:n-1)

                   2
a_0 + a_1*x + a_2*x 

In [7]:
eqset = -Y[1:n] + [y(x => xi) for xi in X[1:n]]

3-element Vector{Sym{PyCall.PyObject}}:
 a_0 + a_1*x1 + a_2*x1^2 - y1
 a_0 + a_1*x2 + a_2*x2^2 - y2
 a_0 + a_1*x3 + a_2*x3^2 - y3

In [8]:
sol = solve(eqset, A[1:n])

Dict{Sym{PyCall.PyObject}, Sym{PyCall.PyObject}} with 3 entries:
  a_2 => (-x1*y2 + x1*y3 + x2*y1 - x2*y3 - x3*y1 + x3*y2)/(x1^2*x2 - x1^2*x3 - …
  a_0 => (x1^2*x2*y3 - x1^2*x3*y2 - x1*x2^2*y3 + x1*x3^2*y2 + x2^2*x3*y1 - x2*x…
  a_1 => (x1^2*y2 - x1^2*y3 - x2^2*y1 + x2^2*y3 + x3^2*y1 - x3^2*y2)/(x1^2*x2 -…

In [9]:
H = [xi^(i - 1) for xi in X[1:n], i in 1:n]

3×3 Matrix{Sym{PyCall.PyObject}}:
 1  x1  x1^2
 1  x2  x2^2
 1  x3  x3^2

In [10]:
solA = H\Y[1:n]
simplify.(solA)

3-element Vector{Sym{PyCall.PyObject}}:
 (x1^2*x2*y3 - x1^2*x3*y2 - x1*x2^2*y3 + x1*x3^2*y2 + x2^2*x3*y1 - x2*x3^2*y1)/(x1^2*x2 - x1^2*x3 - x1*x2^2 + x1*x3^2 + x2^2*x3 - x2*x3^2)
                   (x1^2*y2 - x1^2*y3 - x2^2*y1 + x2^2*y3 + x3^2*y1 - x3^2*y2)/(x1^2*x2 - x1^2*x3 - x1*x2^2 + x1*x3^2 + x2^2*x3 - x2*x3^2)
                              (-x1*y2 + x1*y3 + x2*y1 - x2*y3 - x3*y1 + x3*y2)/(x1^2*x2 - x1^2*x3 - x1*x2^2 + x1*x3^2 + x2^2*x3 - x2*x3^2)

## Lagrange's interpolating polynomial

In [4]:
function l(x, X, j)
    n = length(X)
    out = 1
    for m = 1:n
        if m != j
            out *= (x - X[m])/( X[j] - X[m] )
        end 
    end
    return out
end;

In [9]:
yy = l(x, X[1:2],1)*y1 + l(x, X[1:2],2)*y2 |> expand |> simplify

x*y1 - x*y2 + x1*y2 - x2*y1
---------------------------
          x1 - x2          

In [10]:
yy(x=>0)

x1*y2 - x2*y1
-------------
   x1 - x2   

In [11]:
diff(yy,x)

y1 - y2
-------
x1 - x2

In [ ]:
n = 3
sum( [ Y[j]*l(x,X[1:n],j) for j in 1:n ] )

y1*(x - x2)*(x - x3)   y2*(x - x1)*(x - x3)    y3*(x - x1)*(x - x2)
-------------------- + -------------------- + ---------------------
(x1 - x2)*(x1 - x3)    (-x1 + x2)*(x2 - x3)   (-x1 + x3)*(-x2 + x3)